In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## [WITHOUT] Output Parser

In [2]:
# from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

In [3]:
MODEL_GPT = 'gpt-4o-mini'

In [4]:
# llm = OpenAI()
llm = ChatOpenAI(model=MODEL_GPT)

In [5]:
template_for_desired_output = """
Here is a book review: {book_review}.

I want you to output three things about the review in a JSON dictionary:

"sentiment": Is it positive or negative?
"positive": What is positively highlighted about the book in the review?
"negative": What is negatively highlighted about the book in the review?

"""

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
prompt_template = PromptTemplate(
    input_variables=["book_review"],
    template=template_for_desired_output
)

In [8]:
user_input = """
I bought the kindle edition of this book and I found it to be 
a terrible reading experience. Quote often it seemed like 
pieces of sentences were missing and occasionally when going 
from one page to another I would see the text of two lines 
superimposed on each other. I don't have a kindle so I used 
the android kindle app and kindle for the PC and both left 
much to be desired. Also I hated that if I flipped a few 
pages ahead that it would reset the location. Overall I would 
not recommend the e-book edition, the quality is horrendous. 
Also now that I see the prices, I paid more for the terrible 
kindle edition than the hardcover edition goes for. Anyway 
I would never recommend paying more for an e-book than a print 
book because you have less rights than with the print book. 
Also for a book like this I think there is value to being able 
to flip through it really quickly while the kindle interface 
is best for flipping through page by page.

That being said, the quality of the book was great. It was full 
of all sorts of insights and experiences. They can all be 
summarized as don't give up, watch out for VC's but don't 
write them off, listen to your customers, be willing to change, 
make sure the initial founding team works together well, etc... 
But just listening the values does not do it justice. You 
really have to read the experiences. The book is full of all 
sorts of insights too, not just about entrepreneurship but 
also about the individual companies. For example I was really 
impressed about PayPal and the fraud stuff they did and how 
valuable that was. I just never knew. Overall I think the book 
was very well put together. Although some of the founders liked 
to talk a lot more than others and it droned on and on. But 
others were brief and insightful. I would definitely recommend 
this.

If I bought the print edition I suspect I would be giving it 
5 stars. But really the kindle experience is probably worth 
0 stars. But the content is so good that I figure 4 stars is 
fair. Since at this time I see hardcover editions for $5 or 
$6 new I would say go grab one of those now!!! The book is 
definitely inspirational.
"""

In [9]:
question = prompt_template.format(book_review=user_input)

In [10]:
# response = llm(question)
response = llm.invoke(question)

In [11]:
print(response)

content='```json\n{\n  "sentiment": "mixed",\n  "positive": [\n    "The book is full of insights and experiences.",\n    "It emphasizes important values such as not giving up, listening to customers, and being willing to change.",\n    "The book is well put together and has valuable content about individual companies.",\n    "The content is inspirational."\n  ],\n  "negative": [\n    "The Kindle edition has a terrible reading experience with missing sentences and text superimposed.",\n    "The Kindle app resets location when flipping pages ahead.",\n    "The e-book edition was more expensive than the hardcover edition.",\n    "The reading interface is not conducive to quickly flipping through pages."\n  ]\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 554, 'total_tokens': 698, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0

In [12]:
print(response.content)

```json
{
  "sentiment": "mixed",
  "positive": [
    "The book is full of insights and experiences.",
    "It emphasizes important values such as not giving up, listening to customers, and being willing to change.",
    "The book is well put together and has valuable content about individual companies.",
    "The content is inspirational."
  ],
  "negative": [
    "The Kindle edition has a terrible reading experience with missing sentences and text superimposed.",
    "The Kindle app resets location when flipping pages ahead.",
    "The e-book edition was more expensive than the hardcover edition.",
    "The reading interface is not conducive to quickly flipping through pages."
  ]
}
```


In [13]:
print(type(response))
print(type(response.content))

<class 'langchain_core.messages.ai.AIMessage'>
<class 'str'>


## [WITH] Pydantic Output Parser

In [14]:
from langchain.output_parsers import PydanticOutputParser

In [15]:
# from langchain_core.pydantic_v1 import BaseModel, Field, validator
from pydantic import BaseModel, Field, validator

In [16]:
from typing import List

In [17]:
class Reviews_Desired_Output_Structure(BaseModel):
    sentiment: List[str] = Field(
        description="Is it positive or negative?"
    )
    positive: List[str] = Field(
        description="What is positively highlighted about the book in the review?"
    )
    negative: List[str] = Field(
        description="What is negatively highlighted about the book in the review?"
    )

In [18]:
output_parser = PydanticOutputParser(
    pydantic_object=Reviews_Desired_Output_Structure
)

In [19]:
template_for_desired_output_with_parser = """
Here is a book review: {book_review}.

I want you to output three things about the review in a JSON dictionary:

{format_instructions}

"sentiment": Is it positive or negative?
"positive": What is positively highlighted about the book in the review?
"negative": What is negatively highlighted about the book in the review?

"""

In [20]:
prompt_template_with_parser = PromptTemplate(
    template=template_for_desired_output_with_parser,
    input_variables=["book_review"],
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    }
)

In [21]:
question = prompt_template_with_parser.format(book_review=user_input)

In [22]:
# response = llm(question)
response = llm.invoke(question)

In [23]:
# formatted_output = output_parser.parse(response)
response_text = response.content  # Assuming 'response' is an AIMessage object
formatted_output = output_parser.parse(response_text)

In [24]:
print(type(formatted_output))

<class '__main__.Reviews_Desired_Output_Structure'>


In [25]:
# json_output = formatted_output.json()
json_output = formatted_output.model_dump_json()

In [26]:
import json

In [27]:
python_dict = json.loads(json_output)

In [28]:
print(python_dict)

{'sentiment': ['positive', 'negative'], 'positive': ['The book is full of insights and experiences.', 'It provides valuable lessons about entrepreneurship.', 'The content is inspirational and well put together.'], 'negative': ['The Kindle edition has formatting issues.', 'The reading experience on the Kindle app is terrible.', 'Paid more for the Kindle edition than the hardcover edition.']}


In [29]:
print(type(python_dict))

<class 'dict'>
